In [1]:
# Cell 1 — Phase 2 Setup & Load Data
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json
from pathlib import Path

# ---------------- Paths (relative to notebook location) ----------------
# Try to detect project root - works if notebook is in notebooks/ subdirectory
current_dir = Path.cwd()
if current_dir.name == "notebooks":
    PROJECT_ROOT = current_dir.parent
elif (current_dir / "notebooks").exists():
    PROJECT_ROOT = current_dir
else:
    # Fallback: assume we're in project root
    PROJECT_ROOT = current_dir

DATA_RAW_DIR = PROJECT_ROOT / "data" / "raw"
DATA_PROCESSED_DIR = PROJECT_ROOT / "data" / "processed"
SRC_DIR = PROJECT_ROOT / "src"

# Load settings
settings_path = SRC_DIR / "config/settings.json"
with open(settings_path, "r", encoding="utf-8") as f:
    SETTINGS = json.load(f)

# ---------------- Load Raw Data ----------------
users_df = pd.read_csv(DATA_RAW_DIR / "users_raw.csv")
sessions_df = pd.read_csv(DATA_RAW_DIR / "sessions_raw.csv")
events_df = pd.read_csv(DATA_RAW_DIR / "events_raw.csv")

# ---------------- Quick Overview ----------------
print("Users DF:", users_df.shape)
print(users_df.head())

print("\nSessions DF:", sessions_df.shape)
print(sessions_df.head())

print("\nEvents DF:", events_df.shape)
print(events_df.head())


Users DF: (5000, 8)
  user_id             name                        email      country  \
0  U00001  Patrick Sanchez       jillrhodes@example.net        Benin   
1  U00002   Allen Robinson  hoffmanjennifer@example.net      Mayotte   
2  U00003    Kevin Pacheco        blakeerik@example.com         Guam   
3  U00004     Tyler Rogers     jamesmichael@example.com  Netherlands   
4  U00005   Monica Herrera          smiller@example.net        Niger   

  device_type traffic_source behavior_label   account_created_at  
0      mobile         direct          buyer  2025-10-05 14:25:42  
1     desktop        organic       explorer  2025-09-26 02:22:41  
2      mobile         direct        scanner  2025-08-23 21:35:15  
3     desktop        organic          buyer  2025-07-27 09:27:28  
4     desktop        organic       explorer  2025-07-24 19:14:24  

Sessions DF: (27539, 11)
  session_id user_id           start_time             end_time  duration_sec  \
0    S000001  U00001  2025-07-29 04:46:

In [2]:
# Cell 2 — Data Cleaning & Timestamp Feature Extraction

# ---------------- Convert to datetime ----------------
users_df['account_created_at'] = pd.to_datetime(users_df['account_created_at'])
sessions_df['start_time'] = pd.to_datetime(sessions_df['start_time'])
sessions_df['end_time'] = pd.to_datetime(sessions_df['end_time'])
events_df['timestamp'] = pd.to_datetime(events_df['timestamp'])

# ---------------- Handle missing values ----------------
events_df['element_id'] = events_df['element_id'].fillna('none')
events_df['scroll_depth'] = events_df['scroll_depth'].fillna(0)
events_df['event_value'] = events_df['event_value'].fillna(0)

# ---------------- Remove obvious outliers ----------------
sessions_df = sessions_df[sessions_df['duration_sec'] > 0]
events_df = events_df[events_df['time_on_page'] >= 0]

# ---------------- Extract timestamp features ----------------
# Sessions
sessions_df['start_hour'] = sessions_df['start_time'].dt.hour
sessions_df['start_dayofweek'] = sessions_df['start_time'].dt.dayofweek
sessions_df['start_month'] = sessions_df['start_time'].dt.month
sessions_df['is_weekend'] = sessions_df['start_dayofweek'].isin([5,6]).astype(int)

# Events
events_df['event_hour'] = events_df['timestamp'].dt.hour
events_df['event_dayofweek'] = events_df['timestamp'].dt.dayofweek
events_df['event_month'] = events_df['timestamp'].dt.month
events_df['event_is_weekend'] = events_df['event_dayofweek'].isin([5,6]).astype(int)

print("Data cleaning complete.")
print(f"Sessions DF: {sessions_df.shape}, Events DF: {events_df.shape}")
sessions_df.head(), events_df.head()


Data cleaning complete.
Sessions DF: (27539, 15), Events DF: (220600, 15)


(  session_id user_id          start_time            end_time  duration_sec  \
 0    S000001  U00001 2025-07-29 04:46:03 2025-07-29 04:48:40           157   
 1    S000002  U00001 2025-11-19 07:16:20 2025-11-19 07:18:50           150   
 2    S000003  U00001 2025-07-10 06:31:04 2025-07-10 06:34:02           178   
 3    S000004  U00001 2025-08-26 01:35:39 2025-08-26 01:39:40           241   
 4    S000005  U00002 2025-07-31 04:50:33 2025-07-31 04:55:27           294   
 
    num_events entry_page exit_page device_type traffic_source behavior_label  \
 0           6       P006      P001      mobile         direct          buyer   
 1          10       P001      P006      mobile         direct          buyer   
 2          10       P003      P002      mobile         direct          buyer   
 3           6       P002      P002      mobile         direct          buyer   
 4           5       P006      P001     desktop        organic       explorer   
 
    start_hour  start_dayofweek  sta

In [3]:
# Cell 3 — Feature Engineering (Enhanced with Time & Engagement Features)

# ---------------- Session-level features ----------------
session_features = sessions_df.copy()

# Pages per session
pages_per_session = events_df.groupby("session_id")["page_id"].nunique().reset_index()
pages_per_session.rename(columns={"page_id":"pages_per_session"}, inplace=True)
session_features = session_features.merge(pages_per_session, on="session_id", how="left")

# Create binary columns for event types in events_df
events_df['click'] = (events_df['event_type'] == 'click').astype(int)
events_df['scroll'] = (events_df['event_type'] == 'scroll').astype(int)
events_df['add_to_cart'] = (events_df['event_type'] == 'add_to_cart').astype(int)
events_df['purchase'] = (events_df['event_type'] == 'purchase').astype(int)
events_df['page_view'] = (events_df['event_type'] == 'page_view').astype(int)

# Event counts per session
event_counts = events_df.groupby("session_id")[["click","scroll","add_to_cart","purchase","page_view"]].sum().reset_index()
session_features = session_features.merge(event_counts, on="session_id", how="left")

# Bounce indicator: sessions with <3 events
session_features["is_bounce"] = (session_features["num_events"] < 3).astype(int)

# Conversion funnel features
session_features["cart_abandonment"] = ((session_features["add_to_cart"] > 0) & (session_features["purchase"] == 0)).astype(int)
session_features["conversion_rate"] = (session_features["purchase"] > 0).astype(int)

# Time-based features (aggregate to session level)
session_features["preferred_hour"] = session_features["start_hour"]
session_features["is_weekend_session"] = session_features["is_weekend"]

# ---------------- User-level features ----------------
user_features = session_features.groupby("user_id").agg({
    "duration_sec": ["mean", "std", "min", "max"],
    "num_events": ["mean", "std"],
    "pages_per_session": ["mean", "std"],
    "click": "sum",
    "scroll": "sum",
    "add_to_cart": "sum",
    "purchase": "sum",
    "page_view": "sum",
    "is_bounce": "mean",
    "cart_abandonment": "sum",
    "conversion_rate": "mean",
    "start_hour": lambda x: x.mode()[0] if len(x.mode()) > 0 else x.median(),  # Most common hour
    "is_weekend": "mean",  # Weekend session ratio
    "start_time": ["min", "max", "count"]  # First session, last session, total sessions
}).reset_index()

# Flatten column names
user_features.columns = ['_'.join(col).strip('_') if col[1] else col[0] for col in user_features.columns.values]

# Rename columns for clarity
user_features.rename(columns={
    "duration_sec_mean": "avg_session_duration",
    "duration_sec_std": "std_session_duration",
    "duration_sec_min": "min_session_duration",
    "duration_sec_max": "max_session_duration",
    "num_events_mean": "avg_events_per_session",
    "num_events_std": "std_events_per_session",
    "pages_per_session_mean": "avg_pages_per_session",
    "pages_per_session_std": "std_pages_per_session",
    "click_sum": "total_clicks",
    "scroll_sum": "total_scrolls",
    "add_to_cart_sum": "total_add_to_cart",
    "purchase_sum": "total_purchases",
    "page_view_sum": "total_page_views",
    "is_bounce_mean": "bounce_rate",
    "cart_abandonment_sum": "total_cart_abandonments",
    "conversion_rate_mean": "conversion_rate",
    "start_hour_<lambda>": "preferred_hour",
    "is_weekend_mean": "weekend_session_ratio",
    "start_time_min": "first_session_date",
    "start_time_max": "last_session_date",
    "start_time_count": "total_sessions"
}, inplace=True)

# Calculate additional engagement features
user_features["days_active"] = (pd.to_datetime(user_features["last_session_date"]) - 
                                pd.to_datetime(user_features["first_session_date"])).dt.days + 1
user_features["sessions_per_day"] = user_features["total_sessions"] / user_features["days_active"].clip(lower=1)
user_features["avg_days_between_sessions"] = user_features["days_active"] / user_features["total_sessions"].clip(lower=1)

# Conversion funnel ratios
user_features["click_to_purchase_ratio"] = user_features["total_purchases"] / user_features["total_clicks"].clip(lower=1)
user_features["cart_to_purchase_ratio"] = user_features["total_purchases"] / user_features["total_add_to_cart"].clip(lower=1)
user_features["cart_abandonment_rate"] = user_features["total_cart_abandonments"] / user_features["total_add_to_cart"].clip(lower=1)

# Engagement score (composite metric)
user_features["engagement_score"] = (
    user_features["avg_session_duration"] / 100 +  # Normalize
    user_features["avg_pages_per_session"] / 5 +   # Normalize
    user_features["total_sessions"] / 10 +         # Normalize
    (1 - user_features["bounce_rate"]) * 2
) / 4

# Fill NaN values from division by zero
user_features = user_features.fillna(0)

# Merge categorical features from users_df
user_features = user_features.merge(users_df[["user_id","device_type","traffic_source","behavior_label"]], on="user_id", how="left")

# ---------------- Page-level features ----------------
page_features = events_df.groupby("page_id").agg({
    "event_type":"count",
    "time_on_page":"mean",
    "scroll_depth":"mean",
    "click":"sum"
}).rename(columns={
    "event_type":"page_views",
    "time_on_page":"avg_time_on_page",
    "scroll_depth":"avg_scroll_depth",
    "click":"click_count"
}).reset_index()

# ---------------- Save processed features ----------------
user_features.to_csv(DATA_PROCESSED_DIR / "user_features.csv", index=False)
session_features.to_csv(DATA_PROCESSED_DIR / "session_features.csv", index=False)
page_features.to_csv(DATA_PROCESSED_DIR / "page_features.csv", index=False)

print("Feature engineering complete.")
user_features.head(), session_features.head(), page_features.head()


Feature engineering complete.


(  user_id  avg_session_duration  std_session_duration  min_session_duration  \
 0  U00001            181.500000             41.412558                   150   
 1  U00002            298.000000             50.149776                   219   
 2  U00003            126.750000             59.902003                    67   
 3  U00004            171.000000             65.130638                   110   
 4  U00005            351.857143             88.681882                   258   
 
    max_session_duration  avg_events_per_session  std_events_per_session  \
 0                   241                8.000000                2.309401   
 1                   378                6.285714                2.214670   
 2                   207                7.500000                1.290994   
 3                   263                5.500000                0.577350   
 4                   503                7.714286                3.039424   
 
    avg_pages_per_session  std_pages_per_session  total_clic

In [4]:
# Cell 4 — Feature Analysis & Selection

# ---------------- Feature correlation analysis ----------------
print("="*60)
print("FEATURE CORRELATION ANALYSIS")
print("="*60)

# Select numeric features for correlation
numeric_features = user_features.select_dtypes(include=[np.number]).columns.tolist()
# Exclude ID and date columns
exclude_cols = ['user_id', 'first_session_date', 'last_session_date', 'days_active']
numeric_features = [col for col in numeric_features if col not in exclude_cols]

corr_matrix = user_features[numeric_features].corr()

# Find highly correlated features (|correlation| > 0.9)
high_corr_pairs = []
for i in range(len(corr_matrix.columns)):
    for j in range(i+1, len(corr_matrix.columns)):
        corr_val = corr_matrix.iloc[i, j]
        if abs(corr_val) > 0.9:
            high_corr_pairs.append((corr_matrix.columns[i], corr_matrix.columns[j], corr_val))

if high_corr_pairs:
    print(f"\n⚠️  Found {len(high_corr_pairs)} highly correlated feature pairs (|r| > 0.9):")
    for feat1, feat2, corr in high_corr_pairs[:10]:  # Show first 10
        print(f"   {feat1} <-> {feat2}: {corr:.3f}")
else:
    print("\n✓ No highly correlated features found (all |r| < 0.9)")

# Feature importance by variance
feature_variance = user_features[numeric_features].var().sort_values(ascending=False)
print(f"\nTop 10 features by variance:")
print(feature_variance.head(10))

print("\n" + "="*60)

# ---------------- Normalization & Encoding ----------------
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer

user_df = user_features.copy()

# Select features for ML (exclude date columns and derived features that might cause leakage)
# Keep core features + new engagement features
numeric_cols = [
    # Core features
    "avg_session_duration", "avg_events_per_session", "avg_pages_per_session",
    "total_clicks", "total_scrolls", "total_add_to_cart", "total_purchases", 
    "bounce_rate",
    # New engagement features
    "std_session_duration", "std_events_per_session", "std_pages_per_session",
    "total_page_views", "total_cart_abandonments", "conversion_rate",
    "preferred_hour", "weekend_session_ratio", "total_sessions",
    "sessions_per_day", "avg_days_between_sessions",
    "click_to_purchase_ratio", "cart_to_purchase_ratio", "cart_abandonment_rate",
    "engagement_score"
]

# Only include columns that exist
numeric_cols = [col for col in numeric_cols if col in user_df.columns]

categorical_cols = ["device_type", "traffic_source"]

# Column Transformer: scale numeric, encode categorical
ct = ColumnTransformer([
    ("num", StandardScaler(), numeric_cols),
    ("cat", OneHotEncoder(sparse_output=False, drop='first'), categorical_cols)  # drop='first' to avoid multicollinearity
])

X_user = ct.fit_transform(user_df)

# Feature names
feature_names_num = numeric_cols
feature_names_cat = ct.named_transformers_["cat"].get_feature_names_out(categorical_cols)
feature_names = list(feature_names_num) + list(feature_names_cat)

# Convert to DataFrame
X_user_df = pd.DataFrame(X_user, columns=feature_names)
X_user_df["user_id"] = user_df["user_id"].values
X_user_df["behavior_label"] = user_df["behavior_label"].values

# ---------------- Save final features ----------------
X_user_df.to_csv(DATA_PROCESSED_DIR / "features_user_ml.csv", index=False)

print(f"\n✓ Normalization & encoding complete.")
print(f"✓ Feature matrix shape: {X_user_df.shape}")
print(f"✓ Numeric features: {len(numeric_cols)}")
print(f"✓ Categorical features (encoded): {len(feature_names_cat)}")
print(f"✓ Total ML features: {len(feature_names)}")
print(f"\nFeature matrix ready for ML.")
X_user_df.head()


FEATURE CORRELATION ANALYSIS

⚠️  Found 3 highly correlated feature pairs (|r| > 0.9):
   avg_session_duration <-> max_session_duration: 0.944
   avg_session_duration <-> engagement_score: 0.929
   max_session_duration <-> engagement_score: 0.914

Top 10 features by variance:
max_session_duration         13367.992102
avg_session_duration          7335.108027
min_session_duration          5332.288874
std_session_duration           939.458086
total_clicks                   197.486319
avg_days_between_sessions      126.009676
total_page_views                69.622556
preferred_hour                  40.594089
total_scrolls                   20.974571
total_sessions                   8.906921
dtype: float64


✓ Normalization & encoding complete.
✓ Feature matrix shape: (5000, 30)
✓ Numeric features: 23
✓ Categorical features (encoded): 5
✓ Total ML features: 28

Feature matrix ready for ML.


,avg_session_duration,avg_events_per_session,avg_pages_per_session,total_clicks,total_scrolls,total_add_to_cart,total_purchases,bounce_rate,std_session_duration,std_events_per_session,...,cart_to_purchase_ratio,cart_abandonment_rate,engagement_score,device_type_mobile,device_type_tablet,traffic_source_direct,traffic_source_organic,traffic_source_referral,user_id,behavior_label
0,-0.369743,-0.005979,0.139124,-0.418231,-0.340704,0.487233,-0.425778,-0.2368,-0.444478,-0.181922,...,-0.391223,2.885430,-0.462398,1.0,0.0,1.0,0.0,0.0,U00001,buyer
1,0.990656,-1.223669,-0.533526,-0.062399,0.532784,-0.426215,-0.425778,-0.2368,-0.159390,-0.263716,...,-0.391223,-0.426031,1.045378,0.0,0.0,0.0,1.0,0.0,U00002,explorer
2,-1.009072,-0.361138,-1.273440,-0.774063,-0.340704,-0.426215,-0.425778,-0.2368,0.158816,-1.061250,...,-0.391223,-0.426031,-1.217058,1.0,0.0,1.0,0.0,0.0,U00003,scanner
3,-0.492354,-1.781777,-2.686004,-0.418231,-0.777448,-0.426215,-0.425778,-0.2368,0.329421,-1.677436,...,-0.391223,-0.426031,-0.900588,0.0,0.0,0.0,1.0,0.0,U00004,buyer
4,1.619558,-0.208927,0.273654,0.578099,0.096040,-0.426215,-0.425778,-0.2368,1.097876,0.448406,...,-0.391223,-0.426031,1.720825,0.0,0.0,0.0,1.0,0.0,U00005,explorer


In [6]:
# Cell 5 — Feature Summary & Validation

print("="*60)
print("FEATURE ENGINEERING SUMMARY")
print("="*60)

print(f"\nTotal features created: {len(user_features.columns)}")
print(f"\nFeature categories:")
print(f"  - Core behavioral features: 8")
print(f"  - Time-based features: 3 (preferred_hour, weekend_session_ratio, total_sessions)")
print(f"  - Engagement features: 4 (days_active, sessions_per_day, avg_days_between_sessions, engagement_score)")
print(f"  - Conversion funnel features: 4 (cart_abandonment_rate, conversion_rate, click_to_purchase_ratio, cart_to_purchase_ratio)")
print(f"  - Statistical features: 6 (std, min, max for duration, events, pages)")

print(f"\n✓ All features saved to: {DATA_PROCESSED_DIR / 'features_user_ml.csv'}")
print(f"✓ Ready for clustering analysis in notebook 03")


FEATURE ENGINEERING SUMMARY

Total features created: 32

Feature categories:
  - Core behavioral features: 8
  - Time-based features: 3 (preferred_hour, weekend_session_ratio, total_sessions)
  - Engagement features: 4 (days_active, sessions_per_day, avg_days_between_sessions, engagement_score)
  - Conversion funnel features: 4 (cart_abandonment_rate, conversion_rate, click_to_purchase_ratio, cart_to_purchase_ratio)
  - Statistical features: 6 (std, min, max for duration, events, pages)

✓ All features saved to: c:\Users\ASUS\Desktop\user_behavior_project\data\processed\features_user_ml.csv
✓ Ready for clustering analysis in notebook 03
